<a href="https://www.kaggle.com/code/adamdandi/2023-2025-training-performance-analysis?scriptVersionId=273639100" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# **2023–2025 Training Performance Analysis: Session Delivery Overview**

## **Introduction**

This document provides an overview of the training performance across **2023–2025**. It aims to analyse the number of sessions delivered during this period to understand the overall demand for each course. The analysis supports data-driven planning for future training programs and helps identify areas with higher participation or emerging needs. This aligns with the organisation’s goal to improve training effectiveness and resource allocation.

## **Objectives**

The key objectives of this documentation are to:

* Determine the total number of sessions delivered per course during **2023–2025**.
* Identify trends in session frequency across different training categories.
* Highlight areas of growth or decline to guide future program planning.
* Support decisions on course scheduling, instructor allocation, and resource investment.

Success will be measured by the clarity and accuracy of insights drawn from the data, enabling actionable planning for the next training cycle.

## **Scope**

The analysis covers all training sessions delivered between **January 2023 and December 2025**. It includes course titles, frequency, and participation data where available. The scope is limited to quantitative delivery metrics and does not include qualitative feedback or performance evaluation. All data is sourced from the internal training records and validated for consistency.

## **Methodology**

The analysis follows these key steps:

1. Collect and verify session data from the **2023–2025** training records.
2. Organise courses by category and delivery frequency.
3. Apply filters to identify patterns and variations in demand.
4. Summarise findings in tables and charts for clarity.

This structured approach ensures that the data is complete, accurate, and ready for interpretation.

## **Expected Outcomes**

The expected outcomes of this analysis include:

* A clear summary of session delivery volume for each course.
* Insights into training demand trends across the **three-year period**.
* Recommendations for strategic adjustments in future training plans.

The results will guide planning for the **2026 training calendar**, ensuring efficient use of resources and alignment with learner demand.

In [1]:
import pandas as pd
import warnings

# Suppress "serial value outside limits" warnings
with warnings.catch_warnings(record=True):
    warnings.simplefilter("ignore")

    # Read Excel without date parsing
    data = pd.read_excel(
        "/kaggle/input/trainingdata/TKI-Course-repository.xlsx",
        engine="openpyxl",
        parse_dates=False,  # ✅ correct parameter
        dtype=str            # ✅ read everything as text to avoid Excel date conversion
    )

# Check Data Structure
print("Data Structure:")
print(data.info())

Data Structure:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42450 entries, 0 to 42449
Data columns (total 53 columns):
 #   Column                                                                                                                                                                                                                                                                          Non-Null Count  Dtype 
---  ------                                                                                                                                                                                                                                                                          --------------  ----- 
 0   https://www.linkedin.com/profile/view?id=32701032&authType=NAME_SEARCH&authToken=xxDe&locale=en_US&srchid=1285069551424960135798&srchindex=1&srchtotal=1&trk=vsrp_people_res_name&trkInfo=VSRPsearchId%3A1285069551424960135798%2CVSRPtargetId%3A32701032%2CVSRPcmpt%3Aprimary

### **Data Preparation**

This step creates a structured dataset focused on analyzing course sessions delivered between **2023 and 2025**. The process involves selecting relevant fields, standardizing date formats, and filtering by year to ensure clean and consistent data for analysis.

- **Column Selection:**  
  From the original dataset, nine key columns are retained to capture essential course and scheduling details:  
  **Organizer**, **Facilitator**, **Course Title**, **Course Structure**, **Method**, **Start Date**, **End Date**, **Course City**, and **Course Country**.  
  These columns provide complete coverage of delivery, timing, and location aspects, removing unrelated fields to improve efficiency and clarity.

- **Date Standardization:**  
  A robust multi-pass date parser converts all date values into a consistent format.  
  It handles:
  - Excel serial numbers (e.g., `45250` → valid date).  
  - Day-first formats (`28.09.2025`, `28/09/2025`, `28-09-2025`).  
  - Year-first ISO formats (`2025-09-28`, `2025/09/28`).  
  - Month-first fallbacks (`09/28/2025`).  
  The cleaned results are stored in two new columns — **`Start date std`** and **`End date std`**.

- **Filtering by Year:**  
  The dataset is filtered to include only sessions where either the start or end date falls within **2023, 2024, or 2025**.  
  Any sessions with dates from **2022** are explicitly removed to keep the analysis focused on recent delivery activity.

- **Verification Output:**  
  The script prints the total number of valid rows and displays the first and last 20 records to confirm accuracy after filtering and parsing.

In [2]:
# Use the existing dataframe named `data`
# Keep ONLY these columns
selected_columns = [
    'Organizer',
    'Facilitator',
    'Course title',
    'Course structure',
    'Method',
    'Start date',
    'End date',
    'Course city',
    'Course country',
]

# Create the working dataset named `course`
course = data[selected_columns].copy()

# ---------- Robust multi-pass date parser (handles '.', '/', '-' and Excel serials) ----------
def parse_dates(s: pd.Series) -> pd.Series:
    # Clean once
    s = s.astype('string').str.strip()

    # 1) Excel serials first (accept '45250' and '45250.0'; avoid short numbers like '2025')
    ser_mask = s.str.fullmatch(r'\d{5,}(?:\.0+)?')
    ser = pd.to_numeric(s.where(ser_mask).str.replace(r'\.0+$', '', regex=True), errors='coerce')
    out = pd.to_datetime(ser, unit='d', origin='1899-12-30', errors='coerce')

    # 2) Day-first strings: '28.09.2025', '28/9/2025', '28-09-2025'
    m = out.isna()
    out[m] = pd.to_datetime(s[m], dayfirst=True, errors='coerce', cache=True)

    # 3) Year-first (ISO etc.): '2025-09-28', '2025/09/28'
    m = out.isna()
    out[m] = pd.to_datetime(s[m], yearfirst=True, errors='coerce', cache=True)

    # 4) Last-chance month-first: '09/28/2025'
    m = out.isna()
    out[m] = pd.to_datetime(s[m], errors='coerce', cache=True)

    return out
# --------------------------------------------------------------------------------------------

# Create standardized datetime columns
course['Start date std'] = parse_dates(course['Start date'])
course['End date std']   = parse_dates(course['End date'])

# Keep ONLY rows where Start date std OR End date std is in 2023/2024/2025
valid_years = {2023, 2024, 2025}
mask_years = (
    course['Start date std'].dt.year.isin(valid_years) |
    course['End date std'].dt.year.isin(valid_years)
)
course = course[mask_years].reset_index(drop=True)
# Explicitly exclude any row where Start/End std is in 2022
course = course[~(course['Start date std'].dt.year.eq(2022) | course['End date std'].dt.year.eq(2022))].reset_index(drop=True)

# (Optional) quick look
print("Filtered rows (years 2023–2025 in Start/End std):", len(course))
display(course.head(20))
display(course.tail(20))

Filtered rows (years 2023–2025 in Start/End std): 17356


/tmp/ipykernel_13/665655471.py:30: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  out[m] = pd.to_datetime(s[m], dayfirst=True, errors='coerce', cache=True)
/tmp/ipykernel_13/665655471.py:34: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  out[m] = pd.to_datetime(s[m], yearfirst=True, errors='coerce', cache=True)
/tmp/ipykernel_13/665655471.py:38: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  out[m] = pd.to_datetime(s[m], errors='coerce', cache=True)
/tmp/ipykernel_13/665655471.py:30: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence thi

,Organizer,Facilitator,Course title,Course structure,Method,Start date,End date,Course city,Course country,Start date std,End date std
0,Somu Training,Teodora Gorski,C-BSC,In-house,Face to face,17/11/2023,19/11/2023,Dammam,Saudi Arabia,2023-11-17,2023-11-19
1,TKI,Malek Mohammed Ghazo,C-B,Open,Live,18/04/2023,22/04/2023,NaN,NaN,2023-04-18,2023-04-22
2,TKI,Malek Mohammed Ghazo,C-B,Open,Live,18/04/2023,22/04/2023,NaN,NaN,2023-04-18,2023-04-22
3,TKI,Adrian Brudan,C-KPI,Open,Live,19/09/2023,23/09/2023,NaN,NaN,2023-09-19,2023-09-23
4,TKI,Codruta Mare,C-DV,In-house,Face to face,27/11/2023,29/11/2023,Riyadh,Saudi Arabia,2023-11-27,2023-11-29
5,TKI,Codruta Mare,C-DV,In-house,Face to face,27/11/2024,29/11/2024,Riyadh,Saudi Arabia,2024-11-27,2024-11-29
6,TKI,Mihai Toma,C-BSC,In-house,Live,2023-08-01 00:00:00,2023-12-01 00:00:00,NaN,NaN,2023-08-01,2023-12-01
7,TKI,Mihai Toma,C-BSC,In-house,Live,2023-08-01 00:00:00,2023-12-01 00:00:00,NaN,NaN,2023-08-01,2023-12-01
8,TKI,Mihai Toma,C-BSC,In-house,Live,2023-08-01 00:00:00,2023-12-01 00:00:00,NaN,NaN,2023-08-01,2023-12-01
9,TKI,Mihai Toma,C-BSC,In-house,Live,2023-08-01 00:00:00,2023-12-01 00:00:00,NaN,NaN,2023-08-01,2023-12-01


,Organizer,Facilitator,Course title,Course structure,Method,Start date,End date,Course city,Course country,Start date std,End date std
17336,TKI,Andrea Minelli,C-KPI,In-house,Face to face,28.09.2025,30.09.2025,Riyadh,Saudi Arabia,2025-09-28,2025-09-30
17337,TKI,Andrea Minelli,C-KPI,In-house,Face to face,28.09.2025,30.09.2025,Riyadh,Saudi Arabia,2025-09-28,2025-09-30
17338,TKI,Andrea Minelli,C-KPI,In-house,Face to face,28.09.2025,30.09.2025,Riyadh,Saudi Arabia,2025-09-28,2025-09-30
17339,TKI,Andrea Minelli,C-KPI,In-house,Face to face,28.09.2025,30.09.2025,Riyadh,Saudi Arabia,2025-09-28,2025-09-30
17340,TKI,Andrea Minelli,C-KPI,In-house,Face to face,28.09.2025,30.09.2025,Riyadh,Saudi Arabia,2025-09-28,2025-09-30
17341,TKI,Andrea Minelli,C-KPI,In-house,Face to face,28.09.2025,30.09.2025,Riyadh,Saudi Arabia,2025-09-28,2025-09-30
17342,GIA,Yasser Ghonimy,C-KPIPP,In-house,Face to face,28.9.2025,2.10.2025,Riyadh,Saudi Arabia,2025-09-28,2025-10-02
17343,GIA,Yasser Ghonimy,C-KPIPP,In-house,Face to face,28.9.2025,2.10.2025,Riyadh,Saudi Arabia,2025-09-28,2025-10-02
17344,GIA,Yasser Ghonimy,C-KPIPP,In-house,Face to face,28.9.2025,2.10.2025,Riyadh,Saudi Arabia,2025-09-28,2025-10-02
17345,GIA,Yasser Ghonimy,C-KPIPP,In-house,Face to face,28.9.2025,2.10.2025,Riyadh,Saudi Arabia,2025-09-28,2025-10-02


### Duplicate Check and Participant Count

In this step, we define one session and count how many participant records belong to it. This helps us measure session volume without changing the source data.

#### Session Rule
A session is the unique combination of four columns: **Facilitator**, **Course Title**, **Method**, **Start Date**, and **End Date**.

#### What We Do
1. Find duplicate rows based on the four columns above.  
2. Add a new column **Participants Count**, which stores the size of each group defined by the four columns.  
3. For any row with no duplicate, set **Participants Count** to 1. This column has no missing values.  
4. Produce three checks for quick validation:  
   - **Total rows**: the number of records in the filtered dataset.  
   - **Total participants count**: the sum of *Participants Count* across unique sessions. This equals the number of rows.  
   - **Total sessions**: the number of unique combinations of the four key columns.  
5. Show a short preview so we can confirm the result.

#### Assumptions and Guardrails
- We do not change the data type of any existing column.  
- We only add the **Participants Count** column.  
- All four grouping columns must be present and readable in the dataset.

In [3]:
# Define the session key
key_cols = ['Facilitator', 'Course title', 'Start date', 'End date']

# Count rows per unique session, aligned back to each row
group_sizes = course.groupby(key_cols, dropna=False).size()
course['Participants count'] = course[key_cols].merge(
    group_sizes.rename('Participants count').reset_index(),
    on=key_cols,
    how='left'
)['Participants count']

# Safety check: ensure no missing values in the new column, default to 1 if any appear
course['Participants count'] = course['Participants count'].fillna(1).astype(int)

# Summary numbers
total_rows = len(course)
total_sessions = group_sizes.shape[0]
total_participants = int(group_sizes.sum())  # equals total_rows by construction

# Print summaries
print("Total rows:", total_rows)
print("Total participants count:", total_participants)
print("Total sessions:", total_sessions)

# Preview
print("\nPreview:")
display(course.head(20))   # top 20
display(course.tail(20))   # bottom 20

Total rows: 17356
Total participants count: 17356
Total sessions: 1055

Preview:


,Organizer,Facilitator,Course title,Course structure,Method,Start date,End date,Course city,Course country,Start date std,End date std,Participants count
0,Somu Training,Teodora Gorski,C-BSC,In-house,Face to face,17/11/2023,19/11/2023,Dammam,Saudi Arabia,2023-11-17,2023-11-19,1
1,TKI,Malek Mohammed Ghazo,C-B,Open,Live,18/04/2023,22/04/2023,NaN,NaN,2023-04-18,2023-04-22,2
2,TKI,Malek Mohammed Ghazo,C-B,Open,Live,18/04/2023,22/04/2023,NaN,NaN,2023-04-18,2023-04-22,2
3,TKI,Adrian Brudan,C-KPI,Open,Live,19/09/2023,23/09/2023,NaN,NaN,2023-09-19,2023-09-23,1
4,TKI,Codruta Mare,C-DV,In-house,Face to face,27/11/2023,29/11/2023,Riyadh,Saudi Arabia,2023-11-27,2023-11-29,1
5,TKI,Codruta Mare,C-DV,In-house,Face to face,27/11/2024,29/11/2024,Riyadh,Saudi Arabia,2024-11-27,2024-11-29,1
6,TKI,Mihai Toma,C-BSC,In-house,Live,2023-08-01 00:00:00,2023-12-01 00:00:00,NaN,NaN,2023-08-01,2023-12-01,16
7,TKI,Mihai Toma,C-BSC,In-house,Live,2023-08-01 00:00:00,2023-12-01 00:00:00,NaN,NaN,2023-08-01,2023-12-01,16
8,TKI,Mihai Toma,C-BSC,In-house,Live,2023-08-01 00:00:00,2023-12-01 00:00:00,NaN,NaN,2023-08-01,2023-12-01,16
9,TKI,Mihai Toma,C-BSC,In-house,Live,2023-08-01 00:00:00,2023-12-01 00:00:00,NaN,NaN,2023-08-01,2023-12-01,16


,Organizer,Facilitator,Course title,Course structure,Method,Start date,End date,Course city,Course country,Start date std,End date std,Participants count
17336,TKI,Andrea Minelli,C-KPI,In-house,Face to face,28.09.2025,30.09.2025,Riyadh,Saudi Arabia,2025-09-28,2025-09-30,13
17337,TKI,Andrea Minelli,C-KPI,In-house,Face to face,28.09.2025,30.09.2025,Riyadh,Saudi Arabia,2025-09-28,2025-09-30,13
17338,TKI,Andrea Minelli,C-KPI,In-house,Face to face,28.09.2025,30.09.2025,Riyadh,Saudi Arabia,2025-09-28,2025-09-30,13
17339,TKI,Andrea Minelli,C-KPI,In-house,Face to face,28.09.2025,30.09.2025,Riyadh,Saudi Arabia,2025-09-28,2025-09-30,13
17340,TKI,Andrea Minelli,C-KPI,In-house,Face to face,28.09.2025,30.09.2025,Riyadh,Saudi Arabia,2025-09-28,2025-09-30,13
17341,TKI,Andrea Minelli,C-KPI,In-house,Face to face,28.09.2025,30.09.2025,Riyadh,Saudi Arabia,2025-09-28,2025-09-30,13
17342,GIA,Yasser Ghonimy,C-KPIPP,In-house,Face to face,28.9.2025,2.10.2025,Riyadh,Saudi Arabia,2025-09-28,2025-10-02,14
17343,GIA,Yasser Ghonimy,C-KPIPP,In-house,Face to face,28.9.2025,2.10.2025,Riyadh,Saudi Arabia,2025-09-28,2025-10-02,14
17344,GIA,Yasser Ghonimy,C-KPIPP,In-house,Face to face,28.9.2025,2.10.2025,Riyadh,Saudi Arabia,2025-09-28,2025-10-02,14
17345,GIA,Yasser Ghonimy,C-KPIPP,In-house,Face to face,28.9.2025,2.10.2025,Riyadh,Saudi Arabia,2025-09-28,2025-10-02,14


### Remove duplicates, keep one row per session

We will drop duplicate rows using the session keys, Facilitator, Course title, Start date, End date. We keep the first row of each session. The Participants count column already stores the group size, so the kept row holds the correct count. No other column types change.

In [4]:
# Keys that define one session
key_cols = ['Facilitator', 'Course title', 'Start date', 'End date']

# Remove duplicates, keep the first row per session
course_dedup = course.drop_duplicates(subset=key_cols, keep='first').copy()

# Quick checks
print("Rows before:", len(course))
print("Rows after :", len(course_dedup))
print("Rows removed:", len(course) - len(course_dedup))

print("\nPreview after dedupe:")
display(course_dedup.head())

Rows before: 17356
Rows after : 1055
Rows removed: 16301

Preview after dedupe:


,Organizer,Facilitator,Course title,Course structure,Method,Start date,End date,Course city,Course country,Start date std,End date std,Participants count
0,Somu Training,Teodora Gorski,C-BSC,In-house,Face to face,17/11/2023,19/11/2023,Dammam,Saudi Arabia,2023-11-17,2023-11-19,1
1,TKI,Malek Mohammed Ghazo,C-B,Open,Live,18/04/2023,22/04/2023,NaN,NaN,2023-04-18,2023-04-22,2
3,TKI,Adrian Brudan,C-KPI,Open,Live,19/09/2023,23/09/2023,NaN,NaN,2023-09-19,2023-09-23,1
4,TKI,Codruta Mare,C-DV,In-house,Face to face,27/11/2023,29/11/2023,Riyadh,Saudi Arabia,2023-11-27,2023-11-29,1
5,TKI,Codruta Mare,C-DV,In-house,Face to face,27/11/2024,29/11/2024,Riyadh,Saudi Arabia,2024-11-27,2024-11-29,1


In [5]:
# Check if there are still duplicates after the cleaning process
remaining_duplicates = course_dedup[course_dedup.duplicated(subset=['Facilitator', 'Course title', 'Start date', 'End date'], keep=False)]

# Display the results
if len(remaining_duplicates) == 0:
    print("✅ No duplicates remain. The data is clean.")
else:
    print(f"⚠️ There are still {len(remaining_duplicates)} duplicates remaining.")
    display(remaining_duplicates.head())

✅ No duplicates remain. The data is clean.


In [6]:
print(course_dedup.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1055 entries, 0 to 17342
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Organizer           1055 non-null   object        
 1   Facilitator         1055 non-null   object        
 2   Course title        1055 non-null   object        
 3   Course structure    1047 non-null   object        
 4   Method              1054 non-null   object        
 5   Start date          1055 non-null   object        
 6   End date            1048 non-null   object        
 7   Course city         490 non-null    object        
 8   Course country      490 non-null    object        
 9   Start date std      1054 non-null   datetime64[ns]
 10  End date std        1043 non-null   datetime64[ns]
 11  Participants count  1055 non-null   int64         
dtypes: datetime64[ns](2), int64(1), object(9)
memory usage: 107.1+ KB
None


In [7]:
# Configure the table to scroll
def table(df, height=400):
    display(df.style.set_table_attributes(f'style="display:inline-block;overflow:auto;height:{height}px;width:auto;"').set_table_styles([{
        'selector': '',
        'props': [('border-collapse', 'collapse'),
                  ('margin', '0px')]}]))

# Display the table
table(course_dedup)

,Organizer,Facilitator,Course title,Course structure,Method,Start date,End date,Course city,Course country,Start date std,End date std,Participants count
0,Somu Training,Teodora Gorski,C-BSC,In-house,Face to face,17/11/2023,19/11/2023,Dammam,Saudi Arabia,2023-11-17 00:00:00,2023-11-19 00:00:00,1
1,TKI,Malek Mohammed Ghazo,C-B,Open,Live,18/04/2023,22/04/2023,nan,nan,2023-04-18 00:00:00,2023-04-22 00:00:00,2
3,TKI,Adrian Brudan,C-KPI,Open,Live,19/09/2023,23/09/2023,nan,nan,2023-09-19 00:00:00,2023-09-23 00:00:00,1
4,TKI,Codruta Mare,C-DV,In-house,Face to face,27/11/2023,29/11/2023,Riyadh,Saudi Arabia,2023-11-27 00:00:00,2023-11-29 00:00:00,1
5,TKI,Codruta Mare,C-DV,In-house,Face to face,27/11/2024,29/11/2024,Riyadh,Saudi Arabia,2024-11-27 00:00:00,2024-11-29 00:00:00,1
6,TKI,Mihai Toma,C-BSC,In-house,Live,2023-08-01 00:00:00,2023-12-01 00:00:00,nan,nan,2023-08-01 00:00:00,2023-12-01 00:00:00,16
22,TKI,Alin Sonda,C-EPM,Open,Live,2023-09-01 00:00:00,13/01/2023,nan,nan,2023-09-01 00:00:00,2023-01-13 00:00:00,5
27,TKI,Fadi Fuad Al-Jafari,C-PM,Open,Live,15/1/2023,19/1/2023,nan,nan,2023-01-15 00:00:00,2023-01-19 00:00:00,29
56,Leoron,Alin Sonda,C-KPIPP,In-house,Face to face,15/1/2023,19/1/2023,Muscat,Oman,2023-01-15 00:00:00,2023-01-19 00:00:00,16
72,TKI,Adrian Brudan,C-KPI,Open,Live,16/01/2023,20/01/2023,nan,nan,2023-01-16 00:00:00,2023-01-20 00:00:00,12


In [8]:
# Export cleaned data to Excel file
output_path = '/kaggle/working/Course-Season-2023-2025-Participants.xlsx'
course_dedup.to_excel(output_path, index=False)

print(f"✅ Data successfully exported to: {output_path}")

✅ Data successfully exported to: /kaggle/working/Course-Season-2023-2025-Participants.xlsx


### Duration & Quality Checks

**What this step does (using `course_dedup`):**

1. **Duration math (inclusive):**
   Creates `Duration (days)` as
   `End date std − Start date std + 1`,
   so a one-day course counts as **1** (not 0).

2. **Targeted flags (`suspicious_mask`):**
   Marks rows for review when:

   * `Duration (days) < 1` → end before start (likely parse/data issue), or
   * `Duration (days) > 15` → unusually long course.

3. **Clear outputs for review:**

   * **Rows with suspicious durations:** shows key columns plus `Start date std`, `End date std`, and `Duration (days)`.
   * **Rows with unparsed dates (NaT):** any row where either `Start date std` or `End date std` is missing.

4. **Summary counters:**
   Prints total rows in `course_dedup`, count of **NaT** rows, and count of **suspicious** rows.

In [9]:
# ---- NEXT STEP: duration math, flags, and review tables (use course_dedup) ----
# Assumes `course_dedup` already has parsed columns: 'Start date std' and 'End date std'

# 1) Duration (inclusive) — a one-day course = 1 day
course_dedup['Duration (days)'] = (
    course_dedup['End date std'] - course_dedup['Start date std']
).dt.days + 1

# 2) Targeted flags
#    - Duration < 1  : end before start (likely parsing/data error)
#    - Duration > 15 : unusually long, needs review
suspicious_mask = (course_dedup['Duration (days)'] < 1) | (course_dedup['Duration (days)'] > 15)

# 3) Clear outputs
#    a) Rows with unparsed dates (either std date is NaT)
nat_mask = course_dedup['Start date std'].isna() | course_dedup['End date std'].isna()
nat_table = course_dedup.loc[nat_mask, [
    'Organizer', 'Facilitator', 'Course title', 'Course structure', 'Method', 'Participants count',
    'Start date', 'End date', 'Course city', 'Course country'
]]

#    b) Rows with suspicious durations
suspicious_table = course_dedup.loc[suspicious_mask, [
    'Organizer', 'Facilitator', 'Course title', 'Course structure', 'Method', 'Participants count',
    'Start date', 'End date', 'Start date std', 'End date std',
    'Duration (days)', 'Course city', 'Course country'
]]

# 4) Summary + displays
rows_total = len(course_dedup)
nat_count = nat_mask.sum()
suspicious_count = suspicious_mask.sum()

print(f"Total rows (dedup): {rows_total}")
print(f"Rows with unparsed dates (NaT): {nat_count}")
print(f"Rows with suspicious durations (<1 or >15 days): {suspicious_count}\n")

print(" ROWS WITH SUSPICIOUS DURATION ")
display(suspicious_table)

print("\n ROWS THAT FAILED TO PARSE (NaT) ")
display(nat_table)

Total rows (dedup): 1055
Rows with unparsed dates (NaT): 13
Rows with suspicious durations (<1 or >15 days): 442

 ROWS WITH SUSPICIOUS DURATION 


/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)


,Organizer,Facilitator,Course title,Course structure,Method,Participants count,Start date,End date,Start date std,End date std,Duration (days),Course city,Course country
6,TKI,Mihai Toma,C-BSC,In-house,Live,16,2023-08-01 00:00:00,2023-12-01 00:00:00,2023-08-01,2023-12-01,123.0,NaN,NaN
22,TKI,Alin Sonda,C-EPM,Open,Live,5,2023-09-01 00:00:00,13/01/2023,2023-09-01,2023-01-13,-230.0,NaN,NaN
437,TKI,Ahmed Al-Zahrani,C-KPI,In-house,Face to face,13,2023-10-01 00:00:00,2023-12-01 00:00:00,2023-10-01,2023-12-01,62.0,Jeddah,Saudi Arabia
450,TKI,Mihai Toma,C-KPI,In-house,Face to face,14,30/1/2023,2023-01-02 00:00:00,2023-01-30,2023-01-02,-27.0,Dubai,United Arab Emirates
464,TKI,Nicoleta Niculescu,C-BSC,Open,Live,8,30/01/2023,2023-03-02 00:00:00,2023-01-30,2023-03-02,32.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17071,RHO,Yasser Ghonimy,C-KPI,Open,Live,14,2025-06-09 00:00:00,2025-10-09 00:00:00,2025-06-09,2025-10-09,123.0,NaN,NaN
17085,Vigilance,Dr. Raed Al-Jowder,C-KPI,Open,Live,6,07.09.2025,11.09.2025,2025-07-09,2025-09-11,65.0,NaN,NaN
17091,iLead,Abdulaziz Hussein,C-KPIPP,In-house,Face to face,25,7.9.2025,11.9.2025,2025-07-09,2025-09-11,65.0,Riyadh,Saudi Arabia
17116,MILE,Ahmed Al-Zahrani,C-KPI,In-house,Face to face,11,2025-07-09 00:00:00,2025-09-09 00:00:00,2025-07-09,2025-09-09,63.0,Riyadh,Saudi Arabia



 ROWS THAT FAILED TO PARSE (NaT) 


,Organizer,Facilitator,Course title,Course structure,Method,Participants count,Start date,End date,Course city,Course country
380,TKI,Radu Cocean,KPI-E,In-house,Face to face,15,2023-12-01 00:00:00,NaN,Dammam,Saudi Arabia
577,TKI,Alin Sonda,KPI-EPM-A,In-house,Face to face,11,2023-09-02 00:00:00,NaN,Riyadh,Saudi Arabia
588,TKI,Alin Sonda,KPI-EPM-A,In-house,Face to face,15,2023-08-02 00:00:00,NaN,Riyadh,Saudi Arabia
2366,TKI,Raluca Vintila,KPI-E,In-house,Live,25,2023-04-07 00:00:00,NaN,NaN,NaN
3758,Beyond Paradigm,Farhan Mahmood,KPI-E,In-house,Face to face,8,19/09/2023,NaN,Islamabad,Pakistan
3766,Beyond Paradigm,Farhan Mahmood,KPI-E,In-house,Face to face,20,18/09/2023,NaN,Rawalpindi,Pakistan
6921,TKI,Yasser Ghonimy,C-SBP,Open,Live,1,25/02/2025,29/02/2025,NaN,NaN
7831,TKI,Dr. Ziyad Jawabra,C-PM,Open,Live,11,2024-06-05 00:00:00,10.05/2024,NaN,NaN
7889,TKI,Radu Cocean,C-PM,In-house,Live,13,2024-08-05 00:00:00,12-15/05/2024,NaN,NaN
8118,Vigilance,Dr. Saleh Al-Ansari,C-KPI,In-house,Live,23,19/05/2024,223/05/2024,NaN,NaN


### **Data Cleaning & Validation Plan**

#### **1. Identify Ambiguity**  
For each date in the `suspicious_table`, check if both the **day** and **month** fall between 1 and 12.  
This helps detect potentially ambiguous dates that may have been misinterpreted due to mixed formats (e.g., `03/07` could mean **March 7** or **July 3**).

#### **2. Create a Flag**  
Add a new column that **flags rows with ambiguous dates**.  
This makes it easy to isolate and review problematic entries before applying corrections.

#### **3. Swap Day and Month**  
For all **flagged dates**, swap the **day** and **month** values.  
This corrects possible day/month parsing errors caused by differing date conventions (e.g., **US vs. EU** formats).

#### **4. Recalculate Duration**  
After correcting dates, **recalculate the duration** (e.g., between `start_date` and `end_date`) to ensure logical and consistent time intervals.

#### **5. Show Before & After**  
Produce a **comparison table** displaying both versions:
- Original (ambiguous) data  
- Corrected (cleaned) data  

This visual check verifies the accuracy of the cleaning process before finalizing the dataset.

In [10]:
# Create a copy to work on, preserving the original table
resolved_table = suspicious_table.copy()

print(" Original Suspicious Table ")
print("Notice 'Course B' has a negative duration, indicating a parsing error.")
display(suspicious_table)


# 1. Define a function to check if a date is ambiguous (day and month are 1-12)
def is_date_ambiguous(dt):
    """Returns True if a datetime object's day and month are both between 1 and 12."""
    if pd.isna(dt):
        return False
    return 1 <= dt.day <= 12 and 1 <= dt.month <= 12

# 2. Apply this function to create boolean flags for start and end dates
resolved_table['start_is_ambiguous'] = resolved_table['Start date std'].apply(is_date_ambiguous)
resolved_table['end_is_ambiguous'] = resolved_table['End date std'].apply(is_date_ambiguous)

# Create a single flag for the entire row
resolved_table['Ambiguous Flag'] = resolved_table['start_is_ambiguous'] | resolved_table['end_is_ambiguous']


# 3. Define a function to swap the day and month of a datetime object
def swap_day_month(dt):
    """Returns a new datetime object with the day and month swapped."""
    if pd.isna(dt):
        return pd.NaT
    # Create a new Timestamp to avoid errors with non-existent dates (e.g., 31st of Feb)
    return pd.Timestamp(year=dt.year, month=dt.day, day=dt.month)


# 4. Create new 'resolved' columns
# Initialize them with the original dates
resolved_table['Start date resolved'] = resolved_table['Start date std']
resolved_table['End date resolved'] = resolved_table['End date std']

# Conditionally apply the swap only where the dates were flagged as ambiguous
resolved_table.loc[resolved_table['start_is_ambiguous'], 'Start date resolved'] = resolved_table.loc[resolved_table['start_is_ambiguous'], 'Start date std'].apply(swap_day_month)
resolved_table.loc[resolved_table['end_is_ambiguous'], 'End date resolved'] = resolved_table.loc[resolved_table['end_is_ambiguous'], 'End date std'].apply(swap_day_month)


# 5. Recalculate the duration based on the new 'resolved' dates
new_duration = (resolved_table['End date resolved'] - resolved_table['Start date resolved'])
resolved_table['Duration resolved (days)'] = new_duration.dt.days + 1


# 6. Create the final "Before and After" comparison table for easy review
comparison_view = resolved_table[[
    'Course title',
    'Start date std',
    'End date std',
    'Duration (days)',
    'Ambiguous Flag',
    'Start date resolved',
    'End date resolved',
    'Duration resolved (days)'
]].copy()

# Format dates for clearer display
for col in ['Start date std', 'End date std', 'Start date resolved', 'End date resolved']:
    comparison_view[col] = comparison_view[col].dt.strftime('%Y-%m-%d')


print("\n\n Corrected Dates and Durations (Comparison View) ")
print("The 'resolved' columns show the corrected dates and logical durations.")
display(comparison_view)

 Original Suspicious Table 
Notice 'Course B' has a negative duration, indicating a parsing error.


,Organizer,Facilitator,Course title,Course structure,Method,Participants count,Start date,End date,Start date std,End date std,Duration (days),Course city,Course country
6,TKI,Mihai Toma,C-BSC,In-house,Live,16,2023-08-01 00:00:00,2023-12-01 00:00:00,2023-08-01,2023-12-01,123.0,NaN,NaN
22,TKI,Alin Sonda,C-EPM,Open,Live,5,2023-09-01 00:00:00,13/01/2023,2023-09-01,2023-01-13,-230.0,NaN,NaN
437,TKI,Ahmed Al-Zahrani,C-KPI,In-house,Face to face,13,2023-10-01 00:00:00,2023-12-01 00:00:00,2023-10-01,2023-12-01,62.0,Jeddah,Saudi Arabia
450,TKI,Mihai Toma,C-KPI,In-house,Face to face,14,30/1/2023,2023-01-02 00:00:00,2023-01-30,2023-01-02,-27.0,Dubai,United Arab Emirates
464,TKI,Nicoleta Niculescu,C-BSC,Open,Live,8,30/01/2023,2023-03-02 00:00:00,2023-01-30,2023-03-02,32.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17071,RHO,Yasser Ghonimy,C-KPI,Open,Live,14,2025-06-09 00:00:00,2025-10-09 00:00:00,2025-06-09,2025-10-09,123.0,NaN,NaN
17085,Vigilance,Dr. Raed Al-Jowder,C-KPI,Open,Live,6,07.09.2025,11.09.2025,2025-07-09,2025-09-11,65.0,NaN,NaN
17091,iLead,Abdulaziz Hussein,C-KPIPP,In-house,Face to face,25,7.9.2025,11.9.2025,2025-07-09,2025-09-11,65.0,Riyadh,Saudi Arabia
17116,MILE,Ahmed Al-Zahrani,C-KPI,In-house,Face to face,11,2025-07-09 00:00:00,2025-09-09 00:00:00,2025-07-09,2025-09-09,63.0,Riyadh,Saudi Arabia




 Corrected Dates and Durations (Comparison View) 
The 'resolved' columns show the corrected dates and logical durations.


,Course title,Start date std,End date std,Duration (days),Ambiguous Flag,Start date resolved,End date resolved,Duration resolved (days)
6,C-BSC,2023-08-01,2023-12-01,123.0,True,2023-01-08,2023-01-12,5
22,C-EPM,2023-09-01,2023-01-13,-230.0,True,2023-01-09,2023-01-13,5
437,C-KPI,2023-10-01,2023-12-01,62.0,True,2023-01-10,2023-01-12,3
450,C-KPI,2023-01-30,2023-01-02,-27.0,True,2023-01-30,2023-02-01,3
464,C-BSC,2023-01-30,2023-03-02,32.0,True,2023-01-30,2023-02-03,5
...,...,...,...,...,...,...,...,...
17071,C-KPI,2025-06-09,2025-10-09,123.0,True,2025-09-06,2025-09-10,5
17085,C-KPI,2025-07-09,2025-09-11,65.0,True,2025-09-07,2025-11-09,64
17091,C-KPIPP,2025-07-09,2025-09-11,65.0,True,2025-09-07,2025-11-09,64
17116,C-KPI,2025-07-09,2025-09-09,63.0,True,2025-09-07,2025-09-09,3


### **Course Duration Quality Checks**

#### **1. Unusually Long Courses**  
Identify all courses lasting **more than 15 days**.  
These may suggest data entry mistakes or incorrect end dates (e.g., an extra zero inflating the duration).

#### **2. Illogical Duration**  
Flag any courses with durations **less than 1 day**.  
Such cases often mean the **end date is earlier than the start date**, indicating a possible reversal or input error.

#### **3. Failed to Parse (NaT)**  
Detect rows where **date parsing failed**, resulting in `NaT` (*Not a Time*) values.  
These records represent **invalid or unreadable date formats** that require manual review or correction.

In [11]:
#  Main Logic: Filter for remaining problematic rows 

# 1. Filter for courses with unusually long durations
long_duration_mask = resolved_table['Duration resolved (days)'] > 15
long_duration_table = resolved_table[long_duration_mask]

# 2. Filter for courses with illogical durations (less than 1 day)
short_duration_mask = resolved_table['Duration resolved (days)'] < 1
short_duration_table = resolved_table[short_duration_mask]

# 3. Define columns for a clean, consistent view
review_cols = [
    'Organizer', 'Course title','Facilitator', 'Participants count',
    'Start date std', 'End date std',
    'Start date resolved', 'End date resolved', 'Duration resolved (days)'
]

#  Display Final Actionable Tables 

print("FINAL REVIEW LISTS")
print("Use these tables to perform manual corrections on your source data.")

print("\n\n## 1. Unusually Long Duration (> 15 days)")
print("Check if these long durations are correct or if there's a typo in the year or month.")
if long_duration_table.empty:
    print("--> No courses found with a duration longer than 15 days.")
else:
    display(long_duration_table[review_cols])

print("\n" + "="*50 + "\n") # Separator

print("## 2. Illogical Duration (< 1 day)")
print("The End Date is before the Start Date. This requires manual correction.")
if short_duration_table.empty:
    print("--> No courses found with a negative duration.")
else:
    display(short_duration_table[review_cols])
    
print("\n" + "="*50 + "\n") # Separator

print("## 3. Failed to Parse (NaT Rows)")
print("These rows contain text or formats that could not be converted to dates.")
if nat_table.empty:
    print("--> No rows failed to parse.")
else:
    display(nat_table)

FINAL REVIEW LISTS
Use these tables to perform manual corrections on your source data.


## 1. Unusually Long Duration (> 15 days)
Check if these long durations are correct or if there's a typo in the year or month.


,Organizer,Course title,Facilitator,Participants count,Start date std,End date std,Start date resolved,End date resolved,Duration resolved (days)
775,TKI,C-KPI,Radu Cocean,8,2023-02-21,2023-03-21,2023-02-21,2023-03-21,29
6665,TKI,C-KPI,Amany Fakhry,8,2024-02-05,2024-03-09,2024-05-02,2024-09-03,125
10682,TKI,C-PM,Cristina Mihailoaie,10,2024-04-10,2024-01-11,2024-10-04,2024-11-01,29
11849,TKI,C-EPM,Amany Fakhry,27,2024-10-11,2024-11-12,2024-11-10,2024-12-11,32
13185,TKI,C-SBP,Radu Cocean,5,2025-06-01,2025-01-10,2025-01-06,2025-10-01,269
13625,TKI,C-KPI,Teodora Gorski,12,2025-03-02,2025-02-07,2025-02-03,2025-07-02,150
13823,TKI,KPI-E,Mariham Magdy,16,2025-09-02,2025-02-09,2025-02-09,2025-09-02,206
13839,TKI,KPI-E,Mariham Magdy,14,2025-10-02,2025-02-10,2025-02-10,2025-10-02,235
14073,TKI,C-PM,Dr. Ziyad Jawabra,7,2025-02-03,2025-03-06,2025-03-02,2025-06-03,94
14314,TKI,C-SBP,Dr. Alina Miertoiu,13,2025-07-04,2025-04-11,2025-04-07,2025-11-04,212




## 2. Illogical Duration (< 1 day)
The End Date is before the Start Date. This requires manual correction.


,Organizer,Course title,Facilitator,Participants count,Start date std,End date std,Start date resolved,End date resolved,Duration resolved (days)
1936,TKI,C-BSC,Dr. Ziyad Jawabra,18,2023-05-21,2023-03-05,2023-05-21,2023-05-03,-17
7577,TKI,C-KPI,Alina Miertoiu,21,2026-04-22,2024-04-26,2026-04-22,2024-04-26,-725
10793,Qatar Skills,C-SBP,Dr. Ziyad Jawabra,8,2204-07-10,2024-10-10,2204-10-07,2024-10-10,-65739
11690,TKI,C-EPM,Manhal Al Dakhlallah,24,2024-05-11,2024-11-07,2024-11-05,2024-07-11,-116
11714,TKI,C-KPIPP,Dr. Alina Miertoiu,27,2024-03-11,2024-11-07,2024-11-03,2024-07-11,-114
12428,TKI,C-KPI,Yasser Ghonimy,25,2024-01-12,2024-12-05,2024-12-01,2024-05-12,-202
12477,TKI,C-BSC,Alin Sonda,12,2024-02-12,2024-12-05,2024-12-02,2024-05-12,-203
12532,Vigilance,C-SBP,Dr. Raed Al-Jowder,9,2024-01-12,2024-12-05,2024-12-01,2024-05-12,-202
14978,TKI,C-KPI,Mariham Magdy,2,2025-11-05,2025-04-13,2025-05-11,2025-04-13,-27
15370,TKI,C-KPIPP,Dr. Eng. Malek Mohammed Ghazo,14,2025-01-06,2025-06-04,2025-06-01,2025-04-06,-55




## 3. Failed to Parse (NaT Rows)
These rows contain text or formats that could not be converted to dates.


,Organizer,Facilitator,Course title,Course structure,Method,Participants count,Start date,End date,Course city,Course country
380,TKI,Radu Cocean,KPI-E,In-house,Face to face,15,2023-12-01 00:00:00,NaN,Dammam,Saudi Arabia
577,TKI,Alin Sonda,KPI-EPM-A,In-house,Face to face,11,2023-09-02 00:00:00,NaN,Riyadh,Saudi Arabia
588,TKI,Alin Sonda,KPI-EPM-A,In-house,Face to face,15,2023-08-02 00:00:00,NaN,Riyadh,Saudi Arabia
2366,TKI,Raluca Vintila,KPI-E,In-house,Live,25,2023-04-07 00:00:00,NaN,NaN,NaN
3758,Beyond Paradigm,Farhan Mahmood,KPI-E,In-house,Face to face,8,19/09/2023,NaN,Islamabad,Pakistan
3766,Beyond Paradigm,Farhan Mahmood,KPI-E,In-house,Face to face,20,18/09/2023,NaN,Rawalpindi,Pakistan
6921,TKI,Yasser Ghonimy,C-SBP,Open,Live,1,25/02/2025,29/02/2025,NaN,NaN
7831,TKI,Dr. Ziyad Jawabra,C-PM,Open,Live,11,2024-06-05 00:00:00,10.05/2024,NaN,NaN
7889,TKI,Radu Cocean,C-PM,In-house,Live,13,2024-08-05 00:00:00,12-15/05/2024,NaN,NaN
8118,Vigilance,Dr. Saleh Al-Ansari,C-KPI,In-house,Live,23,19/05/2024,223/05/2024,NaN,NaN


In [12]:
# 0) Bootstrap working copy from previous step (NO re-parsing)
#    Assumes course_dedup already has 'Start date std', 'End date std', and (optionally) 'Duration (days)'
course_clean = course_dedup.copy()

#  1. Consolidate Automatic Fixes into the Main DataFrame 
# First, let's create the 'resolved' columns in your main 'course_clean' DataFrame.
# We'll start by copying the standard parsed dates.
course_clean['Start date resolved'] = course_clean['Start date std']
course_clean['End date resolved'] = course_clean['End date std']
course_clean['Duration resolved (days)'] = course_clean['Duration (days)']

# Now, update 'course_clean' with the automatic swaps you performed in 'resolved_table'.
# The .update() method neatly transfers the corrected data back to your main table.
course_clean.update(resolved_table)


#  2. Manual Input: Define your corrections here 
# Use the DataFrame index as the key.
# Provide the CORRECT 'start' and 'end' dates in 'YYYY-MM-DD' format.
manual_corrections = {
     # Long Duration (> 15 days)
  775: {'start': '2023-02-21', 'end': '2023-03-21'}, # correct
  6665: {'start': '2024-02-05', 'end': '2024-03-09'}, #swapped month in start and end
  10682: {'start': '2024-10-04', 'end': '2024-11-01'}, # correct
  11849: {'start': '2024-11-10', 'end': '2024-11-12'}, #swapped month in end
  13185: {'start': '2025-01-06', 'end': '2025-01-10'}, #swapped month in end
  13625: {'start': '2025-02-03', 'end': '2025-02-07'}, #swapped month in end
  13823: {'start': '2025-02-09', 'end': '2025-02-09'}, #swapped month in end
  13839: {'start': '2025-02-10', 'end': '2025-02-10'}, #swapped month in end
  14073: {'start': '2025-03-02', 'end': '2025-03-06'}, #swapped month in end
  14314: {'start': '2025-04-07', 'end': '2025-04-11'}, #swapped month in end
  14327: {'start': '2025-04-06', 'end': '2025-04-10'}, #swapped month in end
  14823: {'start': '2025-05-04', 'end': '2025-05-08'}, #swapped month in end
  14837: {'start': '2025-05-04', 'end': '2025-05-08'}, #swapped month in end
  14848: {'start': '2025-05-04', 'end': '2025-05-08'}, #swapped month in end
  14856: {'start': '2025-05-04', 'end': '2025-05-08'}, #swapped month in end
  14917: {'start': '2025-05-04', 'end': '2025-05-08'}, #swapped month in end
  15931: {'start': '2025-07-06', 'end': '2025-07-10'}, #swapped month in end
  15948: {'start': '2025-07-06', 'end': '2025-07-08'}, #swapped month in end
  15968: {'start': '2025-07-06', 'end': '2025-07-10'}, #swapped month in end
  16496: {'start': '2025-08-10', 'end': '2025-08-12'}, #swapped month in end
  16522: {'start': '2025-08-10', 'end': '2025-08-12'}, #swapped month in end
  16541: {'start': '2025-08-17', 'end': '2025-08-19'},  # End year was likely 2025, not 2028 and month in end typo, should be 08
  16906: {'start': '2025-09-01', 'end': '2025-09-10'}, #swapped month in end
  16985: {'start': '2025-09-07', 'end': '2025-09-11'}, #swapped month in end
  17011: {'start': '2025-09-08', 'end': '2025-09-12'}, #swapped month in end
  17045: {'start': '2025-09-08', 'end': '2025-09-12'}, #swapped month in end
  17056: {'start': '2025-09-08', 'end': '2025-09-12'}, #swapped month in end
  17085: {'start': '2025-09-07', 'end': '2025-09-11'}, #swapped month in end
  17091: {'start': '2025-09-07', 'end': '2025-09-11'}, #swapped month in end
    
    
    # Illogical Duration (< 1 day)
    1936: {'start': '2023-05-03', 'end': '2023-05-21'}, # date swapped beetween start and end
    7577: {'start': '2024-04-22', 'end': '2024-04-26'}, # year is wrong in start
    10793: {'start': '2024-10-07', 'end': '2024-10-10'}, # year is wrong in start and month swapped in start
    11690: {'start': '2024-11-05', 'end': '2024-11-07'}, #swapped month in end
    11714: {'start': '2024-11-03', 'end': '2024-11-07'}, #swapped month in end
    12428: {'start': '2024-12-01', 'end': '2024-12-05'}, #swapped month in end
    12477: {'start': '2024-12-02', 'end': '2024-12-05'}, #swapped month in end
    12532: {'start': '2024-12-01', 'end': '2024-12-05'}, #swapped month in end
    14978: {'start': '2025-05-11', 'end': '2025-05-13'}, #month in end typo
    15370: {'start': '2025-06-01', 'end': '2025-06-04'}, #swapped month in end
    15384: {'start': '2025-06-02', 'end': '2025-06-03'}, #swapped month in end
    15746: {'start': '2025-06-24', 'end': '2025-06-25'},  # date swapped beetween start and end
    16112: {'start': '2025-07-15', 'end': '2025-07-17'}, #month in end typo
    16234: {'start': '2025-07-20', 'end': '2025-07-27'},  # date swapped beetween start and end
    16381: {'start': '2025-08-03', 'end': '2025-08-07'}, #month in end typo

    # NaT Rows (Failed to Parse)
    6921:    {'start': '2025-02-25', 'end': '2025-02-28'},  # 29/02/2025 is invalid, corrected to 28
    15544: {'start': '2025-06-22', 'end': '2025-06-26'},  # Correcting typo in end date year (20225 -> 2025)
}


#  3. Apply Corrections and Recalculate 

# Get the list of rows we are about to fix for the "Before" view
indices_to_fix = list(manual_corrections.keys())
print(" DATA BEFORE MANUAL FIX ")
# Use course_clean, your actual DataFrame name
display(course_clean.loc[indices_to_fix])

# Loop through the dictionary and apply each correction to your main DataFrame
for index, dates in manual_corrections.items():
    if index in course_clean.index:
        # Convert string dates to datetime objects
        start_date = pd.to_datetime(dates['start'], errors='coerce')
        end_date = pd.to_datetime(dates['end'], errors='coerce')

        # Assign the corrected dates
        course_clean.loc[index, 'Start date resolved'] = start_date
        course_clean.loc[index, 'End date resolved'] = end_date
        
        # Recalculate duration for the fixed row
        duration = (end_date - start_date).days + 1
        course_clean.loc[index, 'Duration resolved (days)'] = duration
    else:
        print(f"Warning: Index {index} not found in DataFrame.")

# Ensure integer dtype (nullable) for the resolved duration column
course_clean['Duration resolved (days)'] = course_clean['Duration resolved (days)'].astype('Int64')

print("\n\n DATA AFTER MANUAL FIX ")
print("Durations are now logical and NaT rows have been populated. ✅")
# Display the same rows from course_clean to see the "After" view
display(course_clean.loc[indices_to_fix])

 DATA BEFORE MANUAL FIX 


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Organizer,Facilitator,Course title,Course structure,Method,Start date,End date,Course city,Course country,Start date std,End date std,Participants count,Duration (days),Start date resolved,End date resolved,Duration resolved (days)
775,TKI,Radu Cocean,C-KPI,In-house,Live,21/02/2023,21/03/2023,NaN,NaN,2023-02-21,2023-03-21,8,29.0,2023-02-21,2023-03-21,29.0
6665,TKI,Amany Fakhry,C-KPI,Open,Live,2024-02-05 00:00:00,2024-03-09 00:00:00,NaN,NaN,2024-02-05,2024-03-09,8,34.0,2024-05-02,2024-09-03,125.0
10682,TKI,Cristina Mihailoaie,C-PM,In-house,Live,2024-04-10 00:00:00,2024-01-11 00:00:00,NaN,NaN,2024-04-10,2024-01-11,10,-89.0,2024-10-04,2024-11-01,29.0
11849,TKI,Amany Fakhry,C-EPM,In-house,Face to face,10.11.2024,12.11.2024,Riyadh,Saudi Arabia,2024-10-11,2024-11-12,27,33.0,2024-11-10,2024-12-11,32.0
13185,TKI,Radu Cocean,C-SBP,Open,Live,06.01.2025,10.01.2025,NaN,NaN,2025-06-01,2025-01-10,5,-141.0,2025-01-06,2025-10-01,269.0
13625,TKI,Teodora Gorski,C-KPI,Open,Live,3.02.2025,7.02.2025,NaN,NaN,2025-03-02,2025-02-07,12,-22.0,2025-02-03,2025-07-02,150.0
13823,TKI,Mariham Magdy,KPI-E,In-house,Face to face,9.02.2025,9.02.2025,Riyadh,Saudi Arabia,2025-09-02,2025-02-09,16,-204.0,2025-02-09,2025-09-02,206.0
13839,TKI,Mariham Magdy,KPI-E,In-house,Face to face,10.02.2025,10.02.2025,Riyadh,Saudi Arabia,2025-10-02,2025-02-10,14,-233.0,2025-02-10,2025-10-02,235.0
14073,TKI,Dr. Ziyad Jawabra,C-PM,Open,Live,2.3.2025,6.03.2025,NaN,NaN,2025-02-03,2025-03-06,7,32.0,2025-03-02,2025-06-03,94.0
14314,TKI,Dr. Alina Miertoiu,C-SBP,Open,Live,7.04.2025,11.04.2025,NaN,NaN,2025-07-04,2025-04-11,13,-83.0,2025-04-07,2025-11-04,212.0




 DATA AFTER MANUAL FIX 
Durations are now logical and NaT rows have been populated. ✅


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,Organizer,Facilitator,Course title,Course structure,Method,Start date,End date,Course city,Course country,Start date std,End date std,Participants count,Duration (days),Start date resolved,End date resolved,Duration resolved (days)
775,TKI,Radu Cocean,C-KPI,In-house,Live,21/02/2023,21/03/2023,NaN,NaN,2023-02-21,2023-03-21,8,29.0,2023-02-21,2023-03-21,29
6665,TKI,Amany Fakhry,C-KPI,Open,Live,2024-02-05 00:00:00,2024-03-09 00:00:00,NaN,NaN,2024-02-05,2024-03-09,8,34.0,2024-02-05,2024-03-09,34
10682,TKI,Cristina Mihailoaie,C-PM,In-house,Live,2024-04-10 00:00:00,2024-01-11 00:00:00,NaN,NaN,2024-04-10,2024-01-11,10,-89.0,2024-10-04,2024-11-01,29
11849,TKI,Amany Fakhry,C-EPM,In-house,Face to face,10.11.2024,12.11.2024,Riyadh,Saudi Arabia,2024-10-11,2024-11-12,27,33.0,2024-11-10,2024-11-12,3
13185,TKI,Radu Cocean,C-SBP,Open,Live,06.01.2025,10.01.2025,NaN,NaN,2025-06-01,2025-01-10,5,-141.0,2025-01-06,2025-01-10,5
13625,TKI,Teodora Gorski,C-KPI,Open,Live,3.02.2025,7.02.2025,NaN,NaN,2025-03-02,2025-02-07,12,-22.0,2025-02-03,2025-02-07,5
13823,TKI,Mariham Magdy,KPI-E,In-house,Face to face,9.02.2025,9.02.2025,Riyadh,Saudi Arabia,2025-09-02,2025-02-09,16,-204.0,2025-02-09,2025-02-09,1
13839,TKI,Mariham Magdy,KPI-E,In-house,Face to face,10.02.2025,10.02.2025,Riyadh,Saudi Arabia,2025-10-02,2025-02-10,14,-233.0,2025-02-10,2025-02-10,1
14073,TKI,Dr. Ziyad Jawabra,C-PM,Open,Live,2.3.2025,6.03.2025,NaN,NaN,2025-02-03,2025-03-06,7,32.0,2025-03-02,2025-03-06,5
14314,TKI,Dr. Alina Miertoiu,C-SBP,Open,Live,7.04.2025,11.04.2025,NaN,NaN,2025-07-04,2025-04-11,13,-83.0,2025-04-07,2025-04-11,5


In [13]:
# Export cleaned data to Excel file
output_path = '/kaggle/working/Course-Season-2023-2025-Clean.xlsx'
course_clean.to_excel(output_path, index=False)

print(f"✅ Data successfully exported to: {output_path}")

✅ Data successfully exported to: /kaggle/working/Course-Season-2023-2025-Clean.xlsx


In [14]:
# Configure the table to scroll
def table(df, height=400):
    display(df.style.set_table_attributes(f'style="display:inline-block;overflow:auto;height:{height}px;width:auto;"').set_table_styles([{
        'selector': '',
        'props': [('border-collapse', 'collapse'),
                  ('margin', '0px')]}]))

# Display the table
table(course_clean)

,Organizer,Facilitator,Course title,Course structure,Method,Start date,End date,Course city,Course country,Start date std,End date std,Participants count,Duration (days),Start date resolved,End date resolved,Duration resolved (days)
0,Somu Training,Teodora Gorski,C-BSC,In-house,Face to face,17/11/2023,19/11/2023,Dammam,Saudi Arabia,2023-11-17 00:00:00,2023-11-19 00:00:00,1,3.000000,2023-11-17 00:00:00,2023-11-19 00:00:00,3
1,TKI,Malek Mohammed Ghazo,C-B,Open,Live,18/04/2023,22/04/2023,nan,nan,2023-04-18 00:00:00,2023-04-22 00:00:00,2,5.000000,2023-04-18 00:00:00,2023-04-22 00:00:00,5
3,TKI,Adrian Brudan,C-KPI,Open,Live,19/09/2023,23/09/2023,nan,nan,2023-09-19 00:00:00,2023-09-23 00:00:00,1,5.000000,2023-09-19 00:00:00,2023-09-23 00:00:00,5
4,TKI,Codruta Mare,C-DV,In-house,Face to face,27/11/2023,29/11/2023,Riyadh,Saudi Arabia,2023-11-27 00:00:00,2023-11-29 00:00:00,1,3.000000,2023-11-27 00:00:00,2023-11-29 00:00:00,3
5,TKI,Codruta Mare,C-DV,In-house,Face to face,27/11/2024,29/11/2024,Riyadh,Saudi Arabia,2024-11-27 00:00:00,2024-11-29 00:00:00,1,3.000000,2024-11-27 00:00:00,2024-11-29 00:00:00,3
6,TKI,Mihai Toma,C-BSC,In-house,Live,2023-08-01 00:00:00,2023-12-01 00:00:00,nan,nan,2023-08-01 00:00:00,2023-12-01 00:00:00,16,123.000000,2023-01-08 00:00:00,2023-01-12 00:00:00,5
22,TKI,Alin Sonda,C-EPM,Open,Live,2023-09-01 00:00:00,13/01/2023,nan,nan,2023-09-01 00:00:00,2023-01-13 00:00:00,5,-230.000000,2023-01-09 00:00:00,2023-01-13 00:00:00,5
27,TKI,Fadi Fuad Al-Jafari,C-PM,Open,Live,15/1/2023,19/1/2023,nan,nan,2023-01-15 00:00:00,2023-01-19 00:00:00,29,5.000000,2023-01-15 00:00:00,2023-01-19 00:00:00,5
56,Leoron,Alin Sonda,C-KPIPP,In-house,Face to face,15/1/2023,19/1/2023,Muscat,Oman,2023-01-15 00:00:00,2023-01-19 00:00:00,16,5.000000,2023-01-15 00:00:00,2023-01-19 00:00:00,5
72,TKI,Adrian Brudan,C-KPI,Open,Live,16/01/2023,20/01/2023,nan,nan,2023-01-16 00:00:00,2023-01-20 00:00:00,12,5.000000,2023-01-16 00:00:00,2023-01-20 00:00:00,5


In [15]:
print(course_clean.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1055 entries, 0 to 17342
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Organizer                 1055 non-null   object        
 1   Facilitator               1055 non-null   object        
 2   Course title              1055 non-null   object        
 3   Course structure          1047 non-null   object        
 4   Method                    1054 non-null   object        
 5   Start date                1055 non-null   object        
 6   End date                  1048 non-null   object        
 7   Course city               490 non-null    object        
 8   Course country            490 non-null    object        
 9   Start date std            1054 non-null   datetime64[ns]
 10  End date std              1043 non-null   datetime64[ns]
 11  Participants count        1055 non-null   int64         
 12  Duration (days)         

## Add Year Column and Compute Yearly Training Statistics

The following code performs several key steps to analyze training performance data by year:

1. **Adds a `Year` column**  
   - Uses `Start date resolved` if available.  
   - Falls back to `End date resolved` if `Start date` is missing.

2. **Computes yearly aggregates**  
   - Total participants per year.  
   - Total number of courses per year.

In [16]:
# Adds a Year column (uses Start date resolved; if missing, falls back to End date resolved).
# Computes total participants per year.
# Computes number of courses per year.

# 1) Add 'Year' column: prefer Start date resolved, else use End date resolved
year_from_start = course_clean['Start date resolved'].dt.year
year_from_end = course_clean['End date resolved'].dt.year

# Use start year where available; otherwise take end year. Keep pandas nullable integer dtype.
course_clean['Year'] = year_from_start.where(year_from_start.notna(), year_from_end).astype('Int64')

# 2) Aggregate by year
#    - total participants per year
#    - total number of courses per year
yearly_stats = (
    course_clean.dropna(subset=['Year'])  # exclude rows where both dates were missing
    .groupby('Year', dropna=False)
    .agg(
        total_participants=('Participants count', 'sum'),
        total_courses=('Year', 'size')
    )
    .reset_index()
    .sort_values('Year')
)

# If you’re in a notebook and want to see it nicely:
display(yearly_stats)

,Year,total_participants,total_courses
0,2023,6234,387
1,2024,6972,422
2,2025,4150,246


## Cohorts & Participants by Method

**Purpose:** Summarize per **Year × Course title × Method**.

**Requires:** `course_clean` with `Year`, `Course title`, `Method`, `Participants count`.

**Logic:**

* Keep methods: **Face to face**, **Live**; drop missing `Year`.
* Group by Year/Course/Method → `cohorts` (row count), `participants` (sum).
* Pivot wide to columns:
  `Cohorts (Face to face)`, `Cohorts (Live)`,
  `Participants (Face to face)`, `Participants (Live)`; fill missing with 0.
* Sort and produce `per_year_combined`.

**Outputs:**

* `per_year_combined` (all years).
* `tables_by_year[YYYY]` (per year).
* Excel: `per_year_course_cohorts_participants.xlsx` (one sheet per year).

**Notes:** Only the two methods are included; rows without `Year` are excluded.

In [17]:
# === Per-year, per-course cohorts & participants by delivery method ===
# Definition:
# - A "cohort" = how many times a course was delivered in a given calendar year.
#   Since each row in `course_clean` is one delivery/session, cohorts are the
#   number of rows per (Year, Course title, Method).
# - "Face to face" and "Live" come from the `Method` column (categorical, fixed values).
# - Year comes from your existing 'Year' column (derived from Start date resolved, else End date resolved).

from IPython.display import display

# Work on a copy to avoid mutating the original
df = course_clean.copy(deep=True)

# Keep only the two valid delivery methods (as per your note)
valid_methods = ["Face to face", "Live"]
df = df[df["Method"].isin(valid_methods)].copy()

# Guard: drop rows where Year is missing (means both dates were missing)
df = df.dropna(subset=["Year"])

# 1) Group to compute cohorts and participants per (Year, Course title, Method)
#    - cohorts = number of deliveries (row count)
#    - participants = sum of Participants count
by_ycm = (
    df.groupby(["Year", "Course title", "Method"], as_index=False)
      .agg(
          cohorts=("Course title", "size"),
          participants=("Participants count", "sum")
      )
)

# 2) Pivot wider so each Method becomes its own pair of columns
wide = (
    by_ycm.pivot_table(
        index=["Year", "Course title"],
        columns="Method",
        values=["cohorts", "participants"],
        aggfunc="sum",
        fill_value=0
    )
)

# 3) Flatten MultiIndex columns and title them exactly as requested
wide.columns = [
    f"{('Cohorts' if m=='cohorts' else 'Participants')} ({meth})"
    for (m, meth) in wide.columns
]
wide = wide.reset_index().sort_values(["Year", "Course title"])

# 4) Ensure all expected columns exist even if a method is missing for some years
expected_cols = [
    "Cohorts (Face to face)", "Cohorts (Live)",
    "Participants (Face to face)", "Participants (Live)"
]
for col in expected_cols:
    if col not in wide.columns:
        wide[col] = 0

# 5) Final combined table (all years)
per_year_combined = wide[
    ["Year", "Course title"]
    + expected_cols
].copy()

# Show the combined table (nicely in notebooks)
display(per_year_combined)

# 6) Split into separate tables per year (drop the Year column inside each sheet)
tables_by_year = {
    int(year): sub.drop(columns="Year").reset_index(drop=True)
    for year, sub in per_year_combined.groupby("Year", dropna=False)
}

# Example: access a specific year's table
# display(tables_by_year[2023].head())

# 7) Write to Excel with one sheet per year (sheet name = the year)
#    If you only want exactly 3 sheets, see the commented block below.
out_path = "per_year_course_cohorts_participants.xlsx"
with pd.ExcelWriter(out_path) as writer:
    for year in sorted(tables_by_year):
        sheet_name = f"{year}"
        tables_by_year[year].to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Excel saved to: {out_path}")

# --- If you specifically want only 3 sheets (e.g., first three chronological years), use this instead:
# first_three_years = sorted(tables_by_year)[:3]
# out_path_3 = "per_year_course_cohorts_participants_first3.xlsx"
# with pd.ExcelWriter(out_path_3) as writer:
#     for year in first_three_years:
#         tables_by_year[year].to_excel(writer, sheet_name=str(year), index=False)
# print(f"Excel (first 3 years) saved to: {out_path_3}")

,Year,Course title,Cohorts (Face to face),Cohorts (Live),Participants (Face to face),Participants (Live)
0,2023,B-E,4,0,92,0
1,2023,BSC-E/Scorecard Development Workshop,1,0,37,0
2,2023,C PM,0,1,0,9
3,2023,C-ASE,1,5,16,47
4,2023,C-B,1,5,19,34
...,...,...,...,...,...,...
77,2025,KPI-M,5,0,215,0
78,2025,"PM, BSC & KPI-M",3,0,89,0
79,2025,"PM, BSC, & KPI-Awareness",1,0,21,0
80,2025,"PM,BSC & KPI Masterclass",2,0,63,0


Excel saved to: per_year_course_cohorts_participants.xlsx


## Summary & Final Output

* **Cohort definition:** count of deliveries per **(Year, Course, Method)**. Methods are fixed to **“Face to face”** and **“Live”**. **Year** comes from *Start date resolved* (fallback: *End date resolved*).
* **Combined table (all years):**
  `Year, Course Name, Cohorts (Face to face), Cohorts (Live), Participants (Face to face), Participants (Live)`.
* **Per-year deliverables (3 sheets per year):**

  1. **`YYYY_CourseMethod`** – per-course cohorts & participants split by method.
  2. **`YYYY_Summaries`** – KPI block (totals, averages, distinct counts, durations) + Monthly and Method breakdowns.
  3. **`YYYY_Tops`** – Top 10 Courses (sessions & participants), Facilitators (sessions & participants), Countries (participants).
* **Global sheets:**
  **`AllYears_CourseMethod`** (full combined table) and **`Yearly_Overview`** (one row per year with key stats & method splits).
* **Data hygiene:** keep only valid methods, coerce participants to numeric, drop missing Year, add Month from resolved dates.
* **Output file:** `Training_Performance_By_Year.xlsx`.

In [18]:
# === Build per-year tables & summaries (3 sheets per year) and global sheets ===
# Definitions:
# - "Cohort" = how many times a course was delivered in a calendar year.
#   Since each row is a delivery/session, cohorts = row count per (Year, Course, Method).
# - Valid delivery methods are exactly ["Face to face", "Live"] from the categorical `Method`.
# - Year comes from 'Year' column (derived from Start date resolved; else End date resolved).
#
# Output:
# - Excel file with:
#   * For each year Y:
#       - "Y_CourseMethod"
#       - "Y_Summaries"
#       - "Y_Tops"
#   * Global sheets:
#       - "AllYears_CourseMethod"
#       - "Yearly_Overview"

import pandas as pd
import numpy as np

# --- 0) Start from a clean working copy and ensure 'Year' exists ---
analysis_df = course_clean.copy()

if 'Year' not in analysis_df.columns:
    # Fallback in case this cell is run standalone
    y_start = analysis_df['Start date resolved'].dt.year
    y_end   = analysis_df['End date resolved'].dt.year
    analysis_df['Year'] = y_start.where(y_start.notna(), y_end).astype('Int64')

# Standardize Course Name (use the acronym map from the previous cell if it exists)
if 'Course Name' not in analysis_df.columns:
    if 'acronym_map' in globals() and isinstance(acronym_map, dict):
        analysis_df['Course Name'] = (
            analysis_df['Course title'].astype(str).str.strip().map(acronym_map)
            .fillna(analysis_df['Course title'].astype(str).str.strip())
        )
    else:
        analysis_df['Course Name'] = analysis_df['Course title'].astype(str).str.strip()

# Keep only the two valid delivery methods
valid_methods = ["Face to face", "Live"]
analysis_df = analysis_df[analysis_df['Method'].isin(valid_methods)].copy()

# Drop rows without a resolvable Year
analysis_df = analysis_df.dropna(subset=['Year']).copy()

# Ensure numeric 'Participants count'
analysis_df['Participants count'] = pd.to_numeric(analysis_df['Participants count'], errors='coerce').fillna(0)

# Convenience Month column (prefer Start date resolved, else End date resolved)
month_src = analysis_df['Start date resolved'].where(
    analysis_df['Start date resolved'].notna(), analysis_df['End date resolved']
)
analysis_df['Month'] = pd.to_datetime(month_src).dt.to_period('M').astype(str)  # e.g., "2025-01"

# --- 1) Build the combined per-(Year, Course) table with cohorts/participants by Method ---
by_ycm = (
    analysis_df
    .groupby(['Year', 'Course Name', 'Method'], as_index=False)
    .agg(
        cohorts=('Course Name', 'size'),
        participants=('Participants count', 'sum')
    )
)

# Pivot wider so each Method has cohorts + participants columns
wide = by_ycm.pivot_table(
    index=['Year', 'Course Name'],
    columns='Method',
    values=['cohorts', 'participants'],
    aggfunc='sum',
    fill_value=0
)

# Flatten MultiIndex columns with friendly names
wide.columns = [
    f"{'Cohorts' if top=='cohorts' else 'Participants'} ({meth})"
    for (top, meth) in wide.columns
]
wide = wide.reset_index().sort_values(['Year', 'Course Name'])

# Guarantee expected columns exist even if one method is absent
expected_cols = [
    "Cohorts (Face to face)", "Cohorts (Live)",
    "Participants (Face to face)", "Participants (Live)"
]
for col in expected_cols:
    if col not in wide.columns:
        wide[col] = 0

per_year_combined = wide[['Year', 'Course Name'] + expected_cols].copy()

# --- 2) Build a yearly overview summary (one row per year) ---
def _safe_nunique(s):
    return s.dropna().nunique()

def build_yearly_overview(df):
    g = df.groupby('Year', dropna=False)
    out = pd.DataFrame({
        'total_sessions': g.size(),
        'total_participants': g['Participants count'].sum(),
        'distinct_courses': g['Course Name'].apply(_safe_nunique),
        'distinct_facilitators': g['Facilitator'].apply(_safe_nunique),
        'distinct_countries': g['Course country'].apply(_safe_nunique),
        'distinct_cities': g['Course city'].apply(_safe_nunique),
    })
    # Duration stats (if available)
    if 'Duration resolved (days)' in df.columns:
        out['total_duration_days'] = g['Duration resolved (days)'].sum(min_count=1)
        out['avg_duration_days']   = g['Duration resolved (days)'].mean()
    else:
        out['total_duration_days'] = np.nan
        out['avg_duration_days']   = np.nan

    # Method splits
    method_counts = df.groupby(['Year', 'Method']).size().unstack('Method', fill_value=0)
    method_parts  = df.groupby(['Year', 'Method'])['Participants count'].sum().unstack('Method', fill_value=0)
    for meth in valid_methods:
        out[f'sessions_{meth}'] = method_counts.get(meth, 0)
        out[f'participants_{meth}'] = method_parts.get(meth, 0)

    # Averages
    out['avg_participants_per_session'] = (out['total_participants'] / out['total_sessions']).replace([np.inf, -np.inf], np.nan)
    return out.reset_index().sort_values('Year')

yearly_overview = build_yearly_overview(analysis_df)

# --- 3) Per-year extras for the "Summaries" and "Tops" sheets ---
def per_year_kpis(df_y):
    """Return a small KPI table (Metric/Value) for one year."""
    total_sessions = len(df_y)
    total_participants = int(df_y['Participants count'].sum())
    distinct_courses = df_y['Course Name'].nunique(dropna=True)
    distinct_facilitators = df_y['Facilitator'].nunique(dropna=True)
    distinct_countries = df_y['Course country'].nunique(dropna=True)
    distinct_cities = df_y['Course city'].nunique(dropna=True)

    if 'Duration resolved (days)' in df_y.columns:
        total_duration = df_y['Duration resolved (days)'].sum(skipna=True)
        avg_duration = df_y['Duration resolved (days)'].mean(skipna=True)
    else:
        total_duration = np.nan
        avg_duration = np.nan

    # Method splits
    ses_by_method = df_y.groupby('Method').size()
    part_by_method = df_y.groupby('Method')['Participants count'].sum()
    s_ff = int(ses_by_method.get('Face to face', 0))
    s_lv = int(ses_by_method.get('Live', 0))
    p_ff = int(part_by_method.get('Face to face', 0))
    p_lv = int(part_by_method.get('Live', 0))

    avg_participants_per_session = (total_participants / total_sessions) if total_sessions else np.nan

    data = [
        ('Total sessions (cohorts)', total_sessions),
        ('Total participants', total_participants),
        ('Avg participants per session', round(avg_participants_per_session, 2) if pd.notna(avg_participants_per_session) else np.nan),
        ('Distinct courses', distinct_courses),
        ('Distinct facilitators', distinct_facilitators),
        ('Distinct countries', distinct_countries),
        ('Distinct cities', distinct_cities),
        ('Sessions - Face to face', s_ff),
        ('Sessions - Live', s_lv),
        ('Participants - Face to face', p_ff),
        ('Participants - Live', p_lv),
        ('Total duration (days)', int(total_duration) if pd.notna(total_duration) else np.nan),
        ('Avg duration (days)', round(avg_duration, 2) if pd.notna(avg_duration) else np.nan),
    ]
    return pd.DataFrame(data, columns=['Metric', 'Value'])

def per_year_monthly(df_y):
    """Return monthly delivery & participant summary for one year."""
    src = df_y['Start date resolved'].where(df_y['Start date resolved'].notna(),
                                            df_y['End date resolved'])
    m = pd.to_datetime(src).dt.to_period('M').astype(str)
    tmp = df_y.copy()
    tmp['Month'] = m
    return (tmp
            .groupby('Month', as_index=True)
            .agg(Sessions=('Course Name', 'size'),
                 Participants=('Participants count', 'sum'))
            .sort_index())

def per_year_method(df_y):
    """Return method-level summary for one year."""
    return (df_y
            .groupby('Method', as_index=True)
            .agg(Sessions=('Method', 'size'),
                 Participants=('Participants count', 'sum'))
            .astype(int)
            .sort_values('Sessions', ascending=False))

def top_tables(df_y, top_n=10):
    """Return dict of top tables: courses, facilitators, countries (by sessions/participants)."""
    top_course_sessions = (df_y.groupby('Course Name')
                           .size().sort_values(ascending=False).head(top_n)
                           .rename('Sessions').to_frame())
    top_course_participants = (df_y.groupby('Course Name')['Participants count']
                               .sum().sort_values(ascending=False).head(top_n)
                               .rename('Participants').to_frame())

    top_fac_sessions = (df_y.groupby('Facilitator')
                        .size().sort_values(ascending=False).head(top_n)
                        .rename('Sessions').to_frame())
    top_fac_participants = (df_y.groupby('Facilitator')['Participants count']
                            .sum().sort_values(ascending=False).head(top_n)
                            .rename('Participants').to_frame())

    top_country_participants = (df_y.groupby('Course country')['Participants count']
                                .sum().sort_values(ascending=False).head(top_n)
                                .rename('Participants').to_frame())

    return {
        'Top Courses by Sessions': top_course_sessions,
        'Top Courses by Participants': top_course_participants,
        'Top Facilitators by Sessions': top_fac_sessions,
        'Top Facilitators by Participants': top_fac_participants,
        'Top Countries by Participants': top_country_participants,
    }

# --- 4) Write everything to Excel (3 sheets per year + global sheets) ---
out_path = "Training_Performance_By_Year.xlsx"
years = sorted(int(y) for y in analysis_df['Year'].dropna().unique())

with pd.ExcelWriter(out_path) as writer:
    # Global sheets
    per_year_combined.to_excel(writer, sheet_name='AllYears_CourseMethod', index=False)
    yearly_overview.to_excel(writer, sheet_name='Yearly_Overview', index=False)

    # Per-year sheets
    for y in years:
        df_y = analysis_df[analysis_df['Year'] == y].copy()

        # --- Sheet 1: per-course method table (Y_CourseMethod) ---
        y_table = (per_year_combined[per_year_combined['Year'] == y]
                   .drop(columns='Year')
                   .reset_index(drop=True))
        y_sheet_name = f"{y}_CourseMethod"
        y_table.to_excel(writer, sheet_name=y_sheet_name, index=False)

        # --- Sheet 2: Summaries (KPIs + Monthly + Method) ---
        s_sheet = f"{y}_Summaries"
        kpis = per_year_kpis(df_y)
        kpis.to_excel(writer, sheet_name=s_sheet, index=False, startrow=0, startcol=0)

        # Monthly summary below KPIs
        monthly = per_year_monthly(df_y)
        start_row_monthly = len(kpis) + 2
        monthly.to_excel(writer, sheet_name=s_sheet, startrow=start_row_monthly, startcol=0)

        # Method summary to the right of KPIs
        method_sum = per_year_method(df_y)
        start_col_method = 4  # put it a few columns to the right
        method_sum.to_excel(writer, sheet_name=s_sheet, startrow=0, startcol=start_col_method)

        # --- Sheet 3: Tops (multiple small tables stacked) ---
        t_sheet = f"{y}_Tops"
        tops = top_tables(df_y, top_n=10)
        row_cursor = 0
        for title, tbl in tops.items():
            # Write a small title row, then the table
            # We'll write the title as a single-cell DataFrame for simplicity
            pd.DataFrame({title: []}).to_excel(writer, sheet_name=t_sheet,
                                               index=False, header=True,
                                               startrow=row_cursor, startcol=0)
            row_cursor += 1
            tbl.to_excel(writer, sheet_name=t_sheet, startrow=row_cursor, startcol=0)
            row_cursor += len(tbl) + 2  # space before next block

print(f"Excel saved to: {out_path}")

# Optional: quick peek in notebooks
# display(per_year_combined.head())
# display(yearly_overview)

Excel saved to: Training_Performance_By_Year.xlsx
